In [1]:
TAM_IMG = 200
BATCH_SIZE_LOADER = 32
COLOR_MODE = 'rgb'

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation sin criterio
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,          # Escalado de píxeles al rango [0, 1]
    rotation_range=20,          # Rotación aleatoria de la imagen
    shear_range=0.2,            # Deformación de la imagen
    zoom_range=0.2,             # Zoom aleatorio
    horizontal_flip=True,       # Volteo horizontal
    vertical_flip=True,         # Volteo vertical
    fill_mode='nearest'         # Relleno de píxeles
)

val_datagen = ImageDataGenerator(rescale=1.0/255.0)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)

2024-11-23 19:10:54.981543: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732385454.993120   47180 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732385454.996422   47180 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-23 19:10:55.009110: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Directorios
train_dir = "../Celebrity_Faces_Dataset_Duplicada_SPLITTED/train"
val_dir = "../Celebrity_Faces_Dataset_Duplicada_SPLITTED/val"

COLOR_MODE = 'rgb'
# Crear generadores de datos
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(TAM_IMG, TAM_IMG),
    batch_size=BATCH_SIZE_LOADER,
    color_mode=COLOR_MODE,
    class_mode='categorical'
)

val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size=(TAM_IMG, TAM_IMG),
    batch_size=BATCH_SIZE_LOADER,
    color_mode=COLOR_MODE,
    class_mode='categorical'
)

test_dir = "../Celebrity_Faces_Dataset_Duplicada_SPLITTED/test"

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(TAM_IMG, TAM_IMG),
    batch_size=BATCH_SIZE_LOADER,
    class_mode='categorical',
    shuffle=False
)


Found 1197 images belonging to 17 classes.
Found 341 images belonging to 17 classes.
Found 173 images belonging to 17 classes.


### Modelo

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.regularizers import l2

In [ ]:
input_shape = (TAM_IMG, TAM_IMG, 3)
neuronas_salida = len(train_data.class_indices)
filtros_conv = 32
neuronas_capa_densa = 128
kernel_size = (5, 5)
padding_tipo = 'valid'

# Modelo basico + data augmentation
model_3 = Sequential([
    Input(shape=input_shape),

    Conv2D(filtros_conv, kernel_size, activation='relu', padding=padding_tipo),
    MaxPooling2D((2, 2)),

    Conv2D(filtros_conv*2, kernel_size, activation='relu', padding=padding_tipo),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    Dense(neuronas_capa_densa, activation='relu'),
    Dropout(0.5),
    Dense(neuronas_salida, activation='softmax')
])

model_3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_3.summary()

I0000 00:00:1732385459.812211   47180 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2246 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 196, 196, 32)   │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 98, 98, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 94, 94, 64)     │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 47, 47, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 141376)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    18,096,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 17)             │         2,193 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,152,145 (69.24 MB)

 Trainable params: 18,152,145 (69.24 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
model_3.fit(train_data, validation_data=val_data, epochs=20, batch_size=32)

/home/david/miniconda3/envs/tf_wsl/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20


I0000 00:00:1732385474.005966   47346 service.cc:148] XLA service 0x7fce2800b970 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732385474.006004   47346 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2024-11-23 19:11:14.029525: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1732385474.144660   47346 cuda_dnn.cc:529] Loaded cuDNN version 90300
2024-11-23 19:11:14.539187: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[32,32,196,196]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,3,200,200]{3,2,1,0}, f32[32,3,5,5]{3,2,1,0}, f32[32]{0}), window={size=5x5}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"act

 2/38 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.0391 - loss: 6.2211

I0000 00:00:1732385482.288507   47346 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


12/38 ━━━━━━━━━━━━━━━━━━━━ 4s 161ms/step - accuracy: 0.0624 - loss: 5.8183

2024-11-23 19:11:24.400540: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[13,32,196,196]{3,2,1,0}, u8[0]{0}) custom-call(f32[13,3,200,200]{3,2,1,0}, f32[32,3,5,5]{3,2,1,0}, f32[32]{0}), window={size=5x5}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
2024-11-23 19:11:24.527422: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[13,64,94,94]{3,2,1,0}, u8[0]{0}) custom-call(f32[13,32,98,98]{3,2,1,0}, f32[64,32,5,5]{3,2,1,0}, f32[64]{0}), window={size=5x5}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_con

38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step - accuracy: 0.0662 - loss: 4.3365

/home/david/miniconda3/envs/tf_wsl/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-11-23 19:11:32.640408: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[32,32,196,196]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,3,200,200]{3,2,1,0}, f32[32,3,5,5]{3,2,1,0}, f32[32]{0}), window={size=5x5}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kRelu","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]

38/38 ━━━━━━━━━━━━━━━━━━━━ 24s 384ms/step - accuracy: 0.0662 - loss: 4.3106 - val_accuracy: 0.0587 - val_loss: 2.8315
Epoch 2/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 11s 282ms/step - accuracy: 0.0599 - loss: 2.8317 - val_accuracy: 0.0733 - val_loss: 2.8239
Epoch 3/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 8s 211ms/step - accuracy: 0.0849 - loss: 2.8164 - val_accuracy: 0.0850 - val_loss: 2.8205
Epoch 4/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 9s 229ms/step - accuracy: 0.0791 - loss: 2.8213 - val_accuracy: 0.0997 - val_loss: 2.7923
Epoch 5/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 7s 188ms/step - accuracy: 0.0985 - loss: 2.7963 - val_accuracy: 0.0909 - val_loss: 2.7987
Epoch 6/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 11s 301ms/step - accuracy: 0.0987 - loss: 2.8056 - val_accuracy: 0.1085 - val_loss: 2.7902
Epoch 7/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 9s 221ms/step - accuracy: 0.1000 - loss: 2.7966 - val_accuracy: 0.1085 - val_loss: 2.7732
Epoch 8/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 7s 187ms/step - accuracy: 0.1105 - loss: 2.7692 - val_accuracy: 0.1085 - v

### Resultados

In [7]:
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import numpy as np

In [8]:
y_pred = model_3.predict(test_data)
y_pred_classes = np.argmax(y_pred, axis=1)

y_true = test_data.classes

acc = accuracy_score(y_true, y_pred_classes)
test_loss, test_acc = model_3.evaluate(test_data)

print(f"Accuracy sklearn (accuracy_score): {acc:.4f}")
print(f"Test Accuracy Keras (model.evaluate): {test_acc:.4f}")

/home/david/miniconda3/envs/tf_wsl/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4/6 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step

2024-11-23 19:14:53.717342: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[13,32,196,196]{3,2,1,0}, u8[0]{0}) custom-call(f32[13,3,200,200]{3,2,1,0}, f32[32,3,5,5]{3,2,1,0}, f32[32]{0}), window={size=5x5}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kRelu","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
2024-11-23 19:14:53.810192: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[13,64,94,94]{3,2,1,0}, u8[0]{0}) custom-call(f32[13,32,98,98]{3,2,1,0}, f32[64,32,5,5]{3,2,1,0}, f32[64]{0}), window={size=5x5}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_con

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 182ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - accuracy: 0.1725 - loss: 2.4662
Accuracy sklearn (accuracy_score): 0.1734
Test Accuracy Keras (model.evaluate): 0.1734


Solo con 20 epocas el redimiento parecido al primer modelo  
modelo_1 = 0.1676  
modelo_2 = 0.2948  
modelo_3.1 = 0.1734

In [ ]:
# guardar el modelo acc 0.1734
# model_3.save('../modelos/modelo_1-3_DataAug20_acc_0-17.keras')


A diferencia de los modelos anteriores, este no presenta overfitting (por lo menos tan rapidamente) y aprende mucho más lento

In [10]:
# Lo entrenamos con más épocas
model_3.fit(train_data, validation_data=val_data, epochs=20, batch_size=32)

Epoch 1/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 7s 179ms/step - accuracy: 0.1907 - loss: 2.5157 - val_accuracy: 0.1877 - val_loss: 2.4751
Epoch 2/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 7s 176ms/step - accuracy: 0.1869 - loss: 2.4786 - val_accuracy: 0.1760 - val_loss: 2.4630
Epoch 3/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 7s 175ms/step - accuracy: 0.1910 - loss: 2.5290 - val_accuracy: 0.1730 - val_loss: 2.4654
Epoch 4/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 7s 176ms/step - accuracy: 0.2051 - loss: 2.4672 - val_accuracy: 0.1789 - val_loss: 2.4092
Epoch 5/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 9s 246ms/step - accuracy: 0.1939 - loss: 2.4579 - val_accuracy: 0.2111 - val_loss: 2.4182
Epoch 6/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 7s 175ms/step - accuracy: 0.2004 - loss: 2.4300 - val_accuracy: 0.2053 - val_loss: 2.4167
Epoch 7/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 7s 176ms/step - accuracy: 0.2046 - loss: 2.4252 - val_accuracy: 0.2287 - val_loss: 2.3861
Epoch 8/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 7s 189ms/step - accuracy: 0.2029 - loss: 2.3755 - val_accuracy: 0.

El modelo a dejado de mejorar, ninguna de las 2 loss baja y se ha quedado estancado 

In [12]:
test_loss, test_acc = model_3.evaluate(test_data)
print(f"Test Accuracy Keras (model.evaluate): {test_acc:.4f}")

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.1985 - loss: 2.3055
Test Accuracy Keras (model.evaluate): 0.2254


Aún así a mejorado un poco el primer entrenamiento

Solo con 20 epocas el redimiento parecido al primer modelo  
  
modelo_1 = 0.1676  
modelo_2 = 0.2948  
modelo_3.1 = 0.1734  
modelo_3.2 = 0.2254

In [ ]:
# guardar el modelo acc 0.2254
# model_3.save('../modelos/modelo_1-3_DataAug40_acc_0-23.keras')